In [ ]:
from random import randint
import hashlib

def point_add(P, Q, A, B, q): # dodawanie punktow na krzywej
    try:
        F = GF(q)
        LAMBDA = F((Q[1] - P[1]) / (Q[0] - P[0]))
        Rx = F((B * pow(LAMBDA, 2)) - (P[0] + Q[0]) - A)
        Ry = F((((2*P[0]) + Q[0] + A)*LAMBDA) - (B * pow(LAMBDA, 3)) - P[1])
        Ry1 = F(LAMBDA*(P[0] - Rx) - P[1])
        R = (Rx, Ry)
        return R
    except ZeroDivisionError:
        return("Punkt w nieskonczonosci")

def point_double(P, A, B, q): # podwajanie punktu na krzywej 
    try:
        F = GF(q)
        LAMBDA = F(((3 * pow(P[0],2)) + (2 * A * P[0]) + 1) / (2 * B * P[1]))
        Rx = F((B * pow(LAMBDA, 2)) - (P[0] + P[0]) - A)
        Ry = F(LAMBDA*(P[0] - Rx) - P[1])
        R = (Rx, Ry)
        return R
    except ZeroDivisionError:
        return("Punkt w nieskonczonosci")
    
def Montgomery_ladder(P, k, A, B, q): # drabina Montgomerego
    R0 = P
    R1 = point_double(P, A, B, q)
    K = bin(k)[3:]
    F = GF(q)
    try:
        for i in range(0, len(K)):
            if K[i] == '1':
                R0 = point_add(R1, R0, A, B, q)
                R1 = point_double(R1, A, B, q)
            else:
                R1 = point_add(R0, R1, A, B, q)
                R0 = point_double(R0, A, B, q)
        return R0
    except ZeroDivisionError:
        return("Punkt w nieskonczonosci")

def gen_key(q, A, B, G, r, h): # algorytm generowania kluczy
    k = randint(2, r - 2)
    print("Wylosowana wartosc k = " +str(k))
    Q = Montgomery_ladder(G, k, A, B, q)
    print("Obliczony punkt Q = [k]G = "+str(Q))
    priv = k
    pub = Q
    return priv, pub
    
def verify_key(q, A, B, G, r, h, Q): # algorytm weryfikacji klucza publicznego
    F  = GF(q)
    left = B * (Q[1] ** 2)
    right = (Q[0] ** 3) + (A * (Q[0]**2)) + Q[0]
    if Q == "Punkt w nieskonczonosci":
        print("Klucz zostal niepoprawnie wyznaczony")
        return False
    elif not (Q[0] >= 0 and Q[0] < q and Q[1] >= 0 and Q[1]) < q:
        print("Klucz zostal niepoprawnie wyznaczony")
        return False
    elif not(left == right):
        print("Klucz zostal niepoprawnie wyznaczony")
        return False
    elif not(Montgomery_ladder(Q, r, A, B, q) == "Punkt w nieskonczonosci"):
        print("Klucz zostal niepoprawnie wyznaczony")
        return False
    else:
        print("Klucz zostal poprawnie wyznaczony")
        return True

def gen_signature(q, A, B, G, r, h, k, m, hashtype): # algorytm generowania podpisu
    while True:
        t = 0
        while True:
            d = randint(2, r - 2)
            print("Wylosowana wartosc d = "+str(d))
            dG = Montgomery_ladder(G, d, A, B, q)
            print("Obliczona d - ta krotkonsc punktu G [d]G = "+str(dG))
            x1_prim = int(dG[0])
            print("Wartosc x1 po konwersji na liczbe calkowita x1' = "+str(x1_prim))
            t = x1_prim % r
            print("Obliczona wartosc t = x1' mod r = "+str(t))
            if t != 0:
                break
        if hashtype == 256:
            x = hashlib.sha3_256()
            x.update(m.encode())
            hash = x.hexdigest()
            e = int(hash, 16)
            print("Obliczona wartosc e = H(m) = "+str(e))
        elif hashtype == 384:
            x = hashlib.sha3_384()
            x.update(m.encode())
            hash = x.hexdigest()
            e = int(hash, 16)
            print("Obliczona wartosc e = H(m) = "+str(e))
        elif hashtype == 512:
            x = hashlib.sha3_512()
            x.update(m.encode())
            hash = x.hexdigest()
            e = int(hash, 16)
            print("Obliczona wartosc e = H(m) = "+str(e))
        s = int(inverse_mod(d, r)) * (e + (k * t))
        print("Obliczona wartosc s = d^(-1)(e + kt) mod r = "+ str(s))
        if s != 0:
            print("Wartosc s rozna od 0 wiec zwracamy pare (t,s) = "+str(t)+", "+str(s)+")")
            break
    return t, s

def verify_signature(q, A, B, G, r, h, Q, m, t, s, hashtype): # algorytm weryfikacji podpisu
    if t >= 1 and t <= r-1 and s >= 1 and s <= r - 1:
        print("Liczby t i s sa liczbami calkowitymi z przedzialu [1, r-1]")
        return False
    if hashtype == 256:
        x = hashlib.sha3_256()
        x.update(m.encode())
        hash = x.hexdigest()
        e = int(hash, 16)
        print("Obliczona wartosc e = H(m) = "+str(e))
    elif hashtype == 384:
        x = hashlib.sha3_384()
        x.update(m.encode())
        hash = x.hexdigest()
        e = int(hash, 16)
        print("Obliczona wartosc e = H(m) = "+str(e))
    elif hashtype == 512:
        x = hashlib.sha3_512()
        x.update(m.encode())
        hash = x.hexdigest()
        e = int(hash, 16)
        print("Obliczona wartosc e = H(m) = "+str(e))
    w = int(inverse_mod(s, r))
    print("Obliczona wartosc w = "+str(w))
    u1 = (e * w) % r
    print("Obliczona wartosc u1 = ew mod r = "+str(u1))
    u2 = (t * w) % r
    print("Obliczona wartosc u2 = tw mod r = "+str(u2))
    
    u1G = Montgomery_ladder(G, u1, A, B, q)
    u2Q = Montgomery_ladder(Q, u2, A, B, q)
    if u1G == u2Q:
        X = point_double(u1G, A, B, q)
        print("Obliczona wartosc X = [u1]G + [u2]G = "+str(X))
    else:
        X = point_add(u1G, u2Q, A, B, q)
        print("Obliczona wartosc X = [u1]G + [u2]G = "+str(X))
    
    if X == "Punkt w nieskonczonosci":
        print("Podpis niepoprawny")
        return False
    
    x1_prim = int(X[0])
    print("Skonwertowana wspolrzedna x1 punktu X = (x1, y1) do liczby calkowitej = "+str(x1_prim))
    
    v = x1_prim % r
    print("Obliczona wartosci v = x1' mod r "+str(v))
    if v == t:
        print("Podpis poprawny")
        return True
    else:
        print("Podpis niepoprawny")
        return False
    
def emulate(q, A, B, xp, yp, r, h, hashtype, m):
    G = (xp, yp)
    print("\t *Cialo Fq = F"+str(q))
    print("\t *Wspolczynnik krzywej Montgomerego A = "+str(A))
    print("\t *Wspolczynnik krzywej Montgomerego B = "+str(B))
    print("\t *Generator G = "+str(G))
    print("\t *Rzad punktu G r = "+str(r))
    print("\t *Kofaktor h = "+str(h))
    k, Q = gen_key(p, A, B, G, r, h)
    print("\t *Wygenerowany klucz prywatny k = "+str(k))
    print("\t *Wygenerowany klucz publiczny Q = "+str(Q))
    a = verify_key(p, A, B, G, r, h, Q)
    print("\t *Czy klucz publiczny zostal wygenerowany poprawnie?")
    print(a)
    t, s = gen_signature(q, A, B, G, r, h, k, m, 512)
    print("\t *Wygenerowany podpis:")
    print("\t *t = "+str(t))
    print("\t *s = "+str(s))
    print("\t *Czy podpis zostal wygenerowany poprawnie?")
    result = verify_signature(q, A, B, G, r, h, Q, m, t, s, 512)
    print(result)

m = "Wlazl kotek na plotek"
p  = 231584178474632390847141970017375815706539969331281128078915168015826259280027
A = 109227819117868662154379630994709463087620760173549354774620080728811549442446
B = 181083626284327783189155449171134415752459241718454178217821450715970719404703
xp = 205540637596926080939588577069732701849851032900462825236489613346856509868196
yp = 13469445318266409424259943503393157460987994303297591376850779015390209410127
r = 57896044618658097711785492504343953926622427821974477956437168612140166707283
k = 4
hashtype = 256

emulate(p, A, B, xp, yp, r, k, hashtype, m)
print("\n\n\n================================== \n\n\n")

p = 78804012392788958424558080200287227610159478540930893335896586808491443542994421222828532509769831281613255980613821
A = 43084446704287580749198183926640685106879024079663571069490350441549939090714824528711432404596949773583150952681622
B = 24636905333238095751838232929628022907073022477088325521892826386353143300943934256735395407015933835990303975467522
xp = 41421403332564264323232855221525238433259327952083005519850658217557089757549002991261244105964905577375391811425879
yp = 59007080323550366170694708684739625363120486722444301441358045103932371098217261668284229668799360080541994262630281
r = 19701003098197239606139520050071806902539869635232723333971463865806582044672404177706795309489453252712859931072871
k = 4
hashtype = 384

emulate(p, A, B, xp, yp, r, k, hashtype, m)
print("\n\n\n================================== \n\n\n")
p = 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151
A = 1387052548546967089596717836509159165138611664648356349872506177450678929471146476344179111573279274222475681087437065157537413908052494825506032727369393871
B = 6687002551931088477801330753274381960902717316813784181739181605398424425909736980586425431364941834383773033493985735287517743883314856721424564314033105344
xp = 4723489921619873813673126899072726421857590521866369618184196350509419850405794459306580935686140948387831139339343877780247623865515763881063399366811211765
yp = 5596645682538392250992323803259412710050978015828877784721316249774543941016447417769136715660080443779605552838933219764324308045599668981524685858476444398
r = 1716199415032652428745475199770348304317358825035826352348615864796385795849413752507334670311300527686743823763141036847205604264575229225855609452904078971
k = 4
hashtype = 512
emulate(p, A, B, xp, yp, r, k, hashtype, m)


	 *Cialo Fq = F231584178474632390847141970017375815706539969331281128078915168015826259280027
	 *Wspolczynnik krzywej Montgomerego A = 109227819117868662154379630994709463087620760173549354774620080728811549442446
	 *Wspolczynnik krzywej Montgomerego B = 181083626284327783189155449171134415752459241718454178217821450715970719404703
	 *Generator G = (205540637596926080939588577069732701849851032900462825236489613346856509868196, 13469445318266409424259943503393157460987994303297591376850779015390209410127)
	 *Rzad punktu G r = 57896044618658097711785492504343953926622427821974477956437168612140166707283
	 *Kofaktor h = 4
Wylosowana wartosc k = 13929601074205060120982882390855974033668814433077220328905479162844088878894


In [2]:
a = '0000000000000004d32b56671d7eb98833c49b433c272586bc4a1c8a8970528ffa04b966f9426eb9da3fb8fee5d7a851a1677dfe5ee4a6c59d1a9c86fb83f6b32c18764a8f66083ca2f48d117c434cb839915121491dec91bcfa968e38192e437c6c4d0b522a9c93ee3e96970a704511d11ebb37f7a2510fa64d0a4f2278709345bf0d890b25698d9d172608c9044f25c5e5439e672733a0548362397dd1a81c76242ab72e9ec79d79d6fb57e40eb0a4ff344c06c6b1474ee498c977a4770b27bb6a202baa064c2c90ceba4663e669ee273065a1b9e4e7ef580731a9fc8849a9e8bfe4846d1efce7c3f30e771808fb3d6c07b6a5112223455b5e5bb11e9e6104c92525a1b019e656d822ec71b5d33c1cf5294de30e646313eea49709b45442241f64be58ced12496fe64e5352de46723dfcbac1aa201d6beb9160a8b15e35b40926e3474ae974b008b4a5d96957c0eb7f8101a3e3e029a6c13b93ceb1e0a0c1b0e96d0501612b08cdee38bd34c1523d5caddb2aecfa6413377c27497412aca49dbfc44791b9b35f3de10f4360f27cd888c2f1d0cb4f7798505e755bf76fe001a9dfb6e850c0134db6b5adf402ce73b0c183120819bfa62d60afae3565736ea14d3df651a6f701cf9dd58ac6a444a7544514de10b3ad92f657325ae2fbad23bc16e2a313f0adc77a3af288b4dd24e059a309ca713968e333687f232accb3a204033c15ac89109eee3caff684dc019c8d755aba04f3d6950b563464257e3b4fc5cb73bab81cf7be9acbec2ea16e2c39c4a0183d449bfc9073ad8ad31d5443afe0a9d0cfea60d052810ee7dc89115b8ffa298872c5987f1f186fe757397e0b64bba5ea771d9818c5c31b449b9b68e64843ab6fd4d1a7d03652f793d23b1148140e3009a0d4add84d10cbc1cb77b02edfd4e4053bb1eec2f6ee4563964807686ba19754d1161a421416be8f9240ca719e0de32ee0ffdee23a538c8e36ff27290fe7e1dd1178fefc5eadf26c12abc36daf9c29c5de52ec0981ee2a0e2ada3736c8a10c1dc6ed4b853db14381dfe37a1bbe3e4b877c5950fb155041ae1fd9c75f6db0c615bd8b80b48fd23ab1a3c281ec5ba6c22d85a0e3726281781c975dbcfd1ba567a09c1572fe2b132a8f0ee24084b7cc25d82af8f1be24ad3b3e6a530ec5c3ac4f2f81b2ec231f3459326400cfbc0d7d357cc553da088a859da4437635da895285303e3511077b2e258d902b4d26a24ea6b87d853ddb5c59caca4f6a446dfb57f375928271004526980ae5add34279cc8ca7c516dd1720dbd46c96f7e11cff68c558f74111a239658d873c33cef968d6b10b73d1febb169510a26986f83caa6aaa7bc65b8a23f0eef7dad4a0d506241292136ea27911e001bb7f1ae4210db86a1a67d219ce86a82f406129ac586d0396bc7b01c7fcfacbdab64633d42e7e84ee8c921f6f846ad08d6ca4ecfa25773c4ecb26c0dac7b02006343667ff69b89bba77cd44debbd56c372e75d3d8c3d9a925c1707248376424c3ea138996602667b85d9179cf4f75859cdada87beb67174151f2593a4319449953f51fb9e6399ec7583a28a087c10fe4c800000005a69c9d9978af4e2f075b78ab789364d61b4e94670951054ac6d65a83f2c5a5e4231dbac7db081ba2253a09bbc66d4ec0b1c09deca97f6a49ccdf371960bb427b03b2517f1b9ad9bf0624a8f227457d01673c0077bdfb777572efd61013b16c15beeb83351ca8fd4760791e5c0555fa2bf60a47dd3ac3bc7c877d73bfbfc7423af2af8125117454a5e1f99a96854b5db1254a92ae2951a84bb69844baaaf87e7f'
b = '0000000000000004d32b56671d7eb98833c49b433c272586bc4a1c8a8970528ffa04b966f9426eb9da3fb8fee5d7a851a1677dfe5ee4a6c59d1a9c86fb83f6b32c18764a8f66083ca2f48d117c434cb839915121491dec91bcfa968e38192e437c6c4d0b522a9c93ee3e96970a704511d11ebb37f7a2510fa64d0a4f2278709345bf0d890b25698d9d172608c9044f25c5e5439e672733a0548362397dd1a81c76242ab72e9ec79d79d6fb57e40eb0a4ff344c06c6b1474ee498c977a4770b27bb6a202baa064c2c90ceba4663e669ee273065a1b9e4e7ef580731a9fc8849a9e8bfe4846d1efce7c3f30e771808fb3d6c07b6a5112223455b5e5bb11e9e6104c92525a1b019e656d822ec71b5d33c1cf5294de30e646313eea49709b45442241f64be58ced12496fe64e5352de46723dfcbac1aa201d6beb9160a8b15e35b40926e3474ae974b008b4a5d96957c0eb7f8101a3e3e029a6c13b93ceb1e0a0c1b0e96d0501612b08cdee38bd34c1523d5caddb2aecfa6413377c27497412aca49dbfc44791b9b35f3de10f4360f27cd888c2f1d0cb4f7798505e755bf76fe001a9dfb6e850c0134db6b5adf402ce73b0c183120819bfa62d60afae3565736ea14d3df651a6f701cf9dd58ac6a444a7544514de10b3ad92f657325ae2fbad23bc16e2a313f0adc77a3af288b4dd24e059a309ca713968e333687f232accb3a204033c15ac89109eee3caff684dc019c8d755aba04f3d6950b563464257e3b4fc5cb73bab81cf7be9acbec2ea16e2c39c4a0183d449bfc9073ad8ad31d5443afe0a9d0cfea60d052810ee7dc89115b8ffa298872c5987f1f186fe757397e0b64bba5ea771d9818c5c31b449b9b68e64843ab6fd4d1a7d03652f793d23b1148140e3009a0d4add84d10cbc1cb77b02edfd4e4053bb1eec2f6ee4563964807686ba19754d1161a421416be8f9240ca719e0de32ee0ffdee23a538c8e36ff27290fe7e1dd1178fefc5eadf26c12abc36daf9c29c5de52ec0981ee2a0e2ada3736c8a10c1dc6ed4b853db14381dfe37a1bbe3e4b877c5950fb155041ae1fd9c75f6db0c615bd8b80b48fd23ab1a3c281ec5ba6c22d85a0e3726281781c975dbcfd1ba567a09c1572fe2b132a8f0ee24084b7cc25d82af8f1be24ad3b3e6a530ec5c3ac4f2f81b2ec231f3459326400cfbc0d7d357cc553da088a859da4437635da895285303e3511077b2e258d902b4d26a24ea6b87d853ddb5c59caca4f6a446dfb57f375928271004526980ae5add34279cc8ca7c516dd1720dbd46c96f7e11cff68c558f74111a239658d873c33cef968d6b10b73d1febb169510a26986f83caa6aaa7bc65b8a23f0eef7dad4a0d506241292136ea27911e001bb7f1ae4210db86a1a67d219ce86a82f406129ac586d0396bc7b01c7fcfacbdab64633d42e7e84ee8c921f6f846ad08d6ca4ecfa25773c4ecb26c0dac7b02006343667ff69b89bba77cd44debbd56c372e75d3d8c3d9a925c1707248376424c3ea138996602667b85d9179cf4f75859cdada87beb67174151f2593a4319449953f51fb9e6399ec7583a28a087c10fe4c800000005a69c9d9978af4e2f075b78ab789364d61b4e94670951054ac6d65a83f2c5a5e4231dbac7db081ba2253a09bbc66d4ec0b1c09deca97f6a49ccdf371960bb427b03b2517f1b9ad9bf0624a8f227457d01673c0077bdfb777572efd61013b16c15beeb83351ca8fd4760791e5c0555fa2bf60a47dd3ac3bc7c877d73bfbfc7423af2af8125117454a5e1f99a96854b5db1254a92ae2951a84bb69844baaaf87e7f'
a == b

True

In [34]:
'0000000100000004d32b56671d7eb98833c49b433c272586bc4a1c8a8970528ffa04b966f9426eb931069d6bc9204e1aa6be6735a6cf5b7e6ce6572456914fa263bab3601bdbb1cd9ebc2ae940bfe5ae8ff1c6bb7a294bbac9a6caa390e0d362e6db2ab6183a75a0303316fbf068addb727c300baf9a8397ad4bd41a5a4ba7ee5edc401e375c99f1c28dc61e6a7355d70873d5722c41e33549db712afd9d57534b5efc02651b4c74aada3666ef9810369f54a9809ab6fb7111cc500e35823ba17341b7ae14252186d52775032d0f89beb6df7d2c51c087ec813f5fcb5bcfb4cfae6d0b6ec175b750136ef04affa6499e4b51eff80947d2804c85d3a668ff94bed7b4a77d6acfdc55812ab16171d1b54004b6ee66a0da9f78ae0080747c78a913d6b725e9592626a91a97c552969d238931ce4cf61ce65736e79226db23e9618faf83e54e54954e24cac5a7e0ba78c37423643fabe18f6e061140ddcecc4b82bc9944eb4d32f3231082165adeee9915536008a07a43c78244e508393b4de0217c51507e146a477b1e947fb23869d610b49974f2ff95266adb19ad90558714da5f3f6ecf234b4afa1c2be564ea9e3bbcaaa716f8e8199b57d9363468f2eacb622e44742d029b158ca39aa8300559bfdc450e4038d7e617358d893685157dd917e0fe87f282d202fbcda9f9fca96dc75e16755702e4f89dbd542079826f8ba128d1285d744358090bce8ffb63318c7c8f4c6fd81853815f448124ac1c151a6467122949bb8680de89a0a60e71d69f420b90bec200793b6b0daebf53be6f2c803e3c943cecdd875f37402bed8ed1188e6b2813eef04d50b5192495f26cc614258049d4b8ecc0fd593ac8444debb168c33284399512c7d5e2d463ec21d9ccfa332a3694914ad5f014c7fcca8fa09966bed033321ea89a91ddd05c593884d58d89b4a1a2cab8ba2e0ecf5b798d2f40a346853b197f260f58e3b7ce572d806f081b926034e75b64ebd5ea680000000000000000000000000000000000000000000000000000000000000000b253f222e62a4fd6aa78fe35092dbe16daca4918aa0f4fd4162387bc5469bb6d319f9bfadde79ff5fc43dc3a0b439ede26fa448c3b5260670537f2c8769d8a5581ed3685e51313d1c744c13cd078caf6f88ba522d4bfa4f00b298d65c5b88df0c02e269d91611651d42b8ef6eb7c76b5c89e7a8b1dbaa746432638fc26c79b970d7cf1b47dbc7f0c37581b6bd151fb5c5a9714f58ad3762ea7eecd0b974f502e94bd64eec1cde8cd41747ec85c58c46c83c710fda86720a094548cb0d55ac979111bf3b5afc7a3eb1443bd35b1ea5bcfa5d52b8f745976be8f0c4daec89f9bed438b935e8b212112fef86cc9e4a8801a27098f6531208cff1d697a8fb35d068deeebc4be737a018a7a49ec097909131476aeaba069ba15128c2ce22dc28c060f4981dace28283ab0cb398dfbf9d4528e15221c66dfe7d0a97c26e788db844526789e92f567a74d34e5981fd336d72b4d86213a1bd57dd4134a3a3a10d31ac80c46b599cb836d74cf1d4ec6ef7a685571bba08c7d84cec03280c8a5464e7d16d800000004a80da2194cdf0497bb1af9f5961d60cac572b59a9c592adefc77c839c5d3d983231dbac7db081ba2253a09bbc66d4ec0b1c09deca97f6a49ccdf371960bb427b03b2517f1b9ad9bf0624a8f227457d01673c0077bdfb777572efd61013b16c15beeb83351ca8fd4760791e5c0555fa2bf60a47dd3ac3bc7c877d73bfbfc7423af2af8125117454a5e1f99a96854b5db1254a92ae2951a84bb69844baaaf87e7f'[2263]

'4'

In [30]:
for i in range(0, 34):
    print("out_get_yy"+str(i)+"_y &",end= ' ')

out_get_yy0_y & out_get_yy1_y & out_get_yy2_y & out_get_yy3_y & out_get_yy4_y & out_get_yy5_y & out_get_yy6_y & out_get_yy7_y & out_get_yy8_y & out_get_yy9_y & out_get_yy10_y & out_get_yy11_y & out_get_yy12_y & out_get_yy13_y & out_get_yy14_y & out_get_yy15_y & out_get_yy16_y & out_get_yy17_y & out_get_yy18_y & out_get_yy19_y & out_get_yy20_y & out_get_yy21_y & out_get_yy22_y & out_get_yy23_y & out_get_yy24_y & out_get_yy25_y & out_get_yy26_y & out_get_yy27_y & out_get_yy28_y & out_get_yy29_y & out_get_yy30_y & out_get_yy31_y & out_get_yy32_y & out_get_yy33_y & 

In [3]:
len(bin(1099305873793)[2:])

40

In [34]:
for i in range(0, 34):
    print('pub'+str(i), end = " & ")

pub0 & pub1 & pub2 & pub3 & pub4 & pub5 & pub6 & pub7 & pub8 & pub9 & pub10 & pub11 & pub12 & pub13 & pub14 & pub15 & pub16 & pub17 & pub18 & pub19 & pub20 & pub21 & pub22 & pub23 & pub24 & pub25 & pub26 & pub27 & pub28 & pub29 & pub30 & pub31 & pub32 & pub33 & 

In [8]:
len('100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010110000')

336

In [2]:
latex(313123123123123)

NameError: name 'latex' is not defined

In [19]:
for i in range(0, 34):
    print("RAM("+str(i)+") ", end=' & ')

RAM(0)  & RAM(1)  & RAM(2)  & RAM(3)  & RAM(4)  & RAM(5)  & RAM(6)  & RAM(7)  & RAM(8)  & RAM(9)  & RAM(10)  & RAM(11)  & RAM(12)  & RAM(13)  & RAM(14)  & RAM(15)  & RAM(16)  & RAM(17)  & RAM(18)  & RAM(19)  & RAM(20)  & RAM(21)  & RAM(22)  & RAM(23)  & RAM(24)  & RAM(25)  & RAM(26)  & RAM(27)  & RAM(28)  & RAM(29)  & RAM(30)  & RAM(31)  & RAM(32)  & RAM(33)  & 

In [27]:
for i in range(0, 32):
    print("out_sha256_out_ext_data_"+str(i), end = " & ")

out_sha256_out_ext_data_0 & out_sha256_out_ext_data_1 & out_sha256_out_ext_data_2 & out_sha256_out_ext_data_3 & out_sha256_out_ext_data_4 & out_sha256_out_ext_data_5 & out_sha256_out_ext_data_6 & out_sha256_out_ext_data_7 & out_sha256_out_ext_data_8 & out_sha256_out_ext_data_9 & out_sha256_out_ext_data_10 & out_sha256_out_ext_data_11 & out_sha256_out_ext_data_12 & out_sha256_out_ext_data_13 & out_sha256_out_ext_data_14 & out_sha256_out_ext_data_15 & out_sha256_out_ext_data_16 & out_sha256_out_ext_data_17 & out_sha256_out_ext_data_18 & out_sha256_out_ext_data_19 & out_sha256_out_ext_data_20 & out_sha256_out_ext_data_21 & out_sha256_out_ext_data_22 & out_sha256_out_ext_data_23 & out_sha256_out_ext_data_24 & out_sha256_out_ext_data_25 & out_sha256_out_ext_data_26 & out_sha256_out_ext_data_27 & out_sha256_out_ext_data_28 & out_sha256_out_ext_data_29 & out_sha256_out_ext_data_30 & out_sha256_out_ext_data_31 & 

In [70]:
for i in range(0, 4):
#     print('''in_sha256_node_'''+str(i+4) + ''' <= I & x"'''+hex(4+i)[2:].zfill(4)+'''" & x"8383" & third_level_tree('''+str(2047 - (i*512))+''' downto '''+str(1536 - (i*512))+''') & padding;''')
    print('''signal in_sha256_node_'''+str(i+4) + ''' : std_logic_vector(1023 downto 0) := x"'''+str(256*'0')+'''";"''')

signal in_sha256_node_4 : std_logic_vector(1023 downto 0) := x"0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000";"
signal in_sha256_node_5 : std_logic_vector(1023 downto 0) := x"0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000";"
signal in_sha256_node_6 : std_logic_vector(1023 downto 0) := x"0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000";"
signal in_sha256_node_7 : std_l

In [4]:
"0000000500000004d32b56671d7eb98833c49b433c272586bc4a1c8a8970528ffa04b966f9426eb9dd6235a8bacaebcfbcff9f934707d50424e1dddd07b6b326a990bd774c43ce648d1e74be18f66ce6a4f4f45822eb73692aac7bc2bb595f5d6c81baa6b1aff54a5bfa01919f66ac3d07c0ad2fcdbcc6d3543aaa9de77663c151419577644cf26693fd7b8ad0a628ab9695a90778537ea4a779fa79a6758fa68cdc5318b1da7f7dfb53e5e0e4b76d8fb6c4704e1fb0a269f6731785d353a5f5cf9988eac6a80d07a4b03164be014a6790bac08df8234f92359f3d80bde49f8b7d1508a2631108c7bba10a7ff77943bd54584a87837adf81f772238dbdaa050954a588b23b9e8ea100e9ee7e9f54caae1adc8d49625172851cec720ac834e1a809abba2afac54203aaa2f15ac267baa7e03a2a8338a0079fc560337651890e758f00f4f4f0ee25ccb9d1fe88322e7f21b521432d981f482aaa14cf255f57c273f4b88205bef22b60db48dc7fe633160d489cdddbe4d221785a366882c9436ce8c591207eb3b2490b0a77ddcaa1a33deab0dc44b7b63e7d6ecf0e885e5375ee7b3ed8e9995f54d9d5787e81fd41c19d7b9217d19c5b49721c5165f92d09997cb4daae4d896387ac3ac061f4d952739b79543f04794f45e9c3a40812147029e91b73c96001e7989086ec9d7c577e1297225a5b8a946280a29d6e20ddc8359349582dccf15a5bf45ed4dd6a7c69338a309a4427c8d438b7aa915a7cebf027d3762345d471c870019e52fd8cd882aa5a9472d0f787eff2780ef5af926eea10a8e10699ebb5dbbded577644560abab4b348a2629c6fc3b01fba61a7e76cff95faf66492fbe4fc359d2bd0e407da8f196147131d5b7d3ba754f54db4eb6a656faf429295d1654e1f133feb84377624f312c84ed740fd76f8e627847291f4712ecdd05d7b282ce3b14b68de7bdce98e4e71db87c441aaea904c60270251694110f1e8b459eee8d0265bfaf97b5fe44cdac66f3e6b0eae146f006fdbb98dd3eb86820eaf773d9315a9b2e0ef12acfa7edaa654fe5ee92b2bb3cddc1b857b76cafa9ca2220e0089e65581a94084cd36ea3938ff9c8b6398950a1165e5b3ba112db70e0fabe76bcca66e483d40e7d1ad2eb29299e5afb3f916533142241652cbe7173aab7140598468c0f526ff7612e8b8e919d7c97470327c365920b976be9493c7588473ba2b5b82c692380c110e53c418a43ada2083a0bfc233f6343ec44b8ddf7fc5fee014b7e353782751e9e3bd7f5bd96c1ab9f834eefdd5910c4159c4e62de712d7cf2eb6d73f2f5ec9e8c28d545716a81267f150cf8e0ba56281c7b6b81cf1927c4b8703a260e9a2e7a331e175f613f98d64546c8f076e5d94ec0cc1ff03e089896dbf20d8b92214b1c12ada710c1297355863a4fb7276154c15ea13d3225e55aec14dd993d845b4732cdc66da585f6013ffc9dbe857e133097ef81d8ca9ff720e6c32d56689703c8a143053fc354acce51362d13bb7c0dc26cba945a6c18800a9f6fa96449c94067caabc1caca0935aeb80699b92c34b5b85a95272657c2a059363d0d0033a8dbbba8aadcc3b6adce1116be0c6a6076d91dee21d64d51f327a27eac6dc8bd9c9e2d13d9baf7a1a7594b759e37db2289de8cadab38892b36c8d19036d70fc870219282168ae02f62bbeb1f20722409c40bec5cb8209a328604ff4e099269d614aa89cbeeb83351ca8fd4760791e5c0555fa2bf60a47dd3ac3bc7c877d73bfbfc7423af2af8125117454a5e1f99a96854b5db1254a92ae2951a84bb69844baaaf87e7f".upper()

'0000000500000004D32B56671D7EB98833C49B433C272586BC4A1C8A8970528FFA04B966F9426EB9DD6235A8BACAEBCFBCFF9F934707D50424E1DDDD07B6B326A990BD774C43CE648D1E74BE18F66CE6A4F4F45822EB73692AAC7BC2BB595F5D6C81BAA6B1AFF54A5BFA01919F66AC3D07C0AD2FCDBCC6D3543AAA9DE77663C151419577644CF26693FD7B8AD0A628AB9695A90778537EA4A779FA79A6758FA68CDC5318B1DA7F7DFB53E5E0E4B76D8FB6C4704E1FB0A269F6731785D353A5F5CF9988EAC6A80D07A4B03164BE014A6790BAC08DF8234F92359F3D80BDE49F8B7D1508A2631108C7BBA10A7FF77943BD54584A87837ADF81F772238DBDAA050954A588B23B9E8EA100E9EE7E9F54CAAE1ADC8D49625172851CEC720AC834E1A809ABBA2AFAC54203AAA2F15AC267BAA7E03A2A8338A0079FC560337651890E758F00F4F4F0EE25CCB9D1FE88322E7F21B521432D981F482AAA14CF255F57C273F4B88205BEF22B60DB48DC7FE633160D489CDDDBE4D221785A366882C9436CE8C591207EB3B2490B0A77DDCAA1A33DEAB0DC44B7B63E7D6ECF0E885E5375EE7B3ED8E9995F54D9D5787E81FD41C19D7B9217D19C5B49721C5165F92D09997CB4DAAE4D896387AC3AC061F4D952739B79543F04794F45E9C3A40812147029E91B73C96001E7989086EC9D7C577E1297225A5B8A9

In [46]:
len('a80da2194cdf0497bb1af9f5961d60cac572b59a9c592adefc77c839c5d3d983a69c9d9978af4e2f075b78ab789364d61b4e94670951054ac6d65a83f2c5a5e484e6ba5fe17103d4dbea9f495a750b5c0fc4c6e22d6c711d3a11cb6286d483b6fcf9ee9c828669b1360a3428004ce237ed3222503f097e095eefa5f953a255ca8aadcc3b6adce1116be0c6a6076d91dee21d64d51f327a27eac6dc8bd9c9e2d10d150ca237915d9f263a2bd859ead79875a877540553512594d26c5ae37d6f9f897987c7fa1411e0326cb3a041a172d2ebc7eeaf75331e03aaa1204a6c1835251e1ec8e57c92bbccf48bc2b7ead0f11beb87391142e7901813a7dcc4d2aee4ed775b8f0f9ecbfe0020d21e68817a20c552ecbb3d0478b55c421cf5aac88f237c6c5f72f8c1d25ae94924dbf80d6b515ef0c0a7f4ce07a1bcb4fcd3183586700daf0544a7a93319eb29b456fa50f0b63dc37f84519d34d883ae0d515668f9740cfaa5c728b7679520ad373865503c156b3a0ac5f2d01ef219137ee60f6809d073046068181d8ea950ccc1c9ba21ce1e2d25a59980cd1907cc9900059005a721b8b8ccdf3533d978f7436fbde8f143aa4cd14c4131776a721864ce0bb5a361ef482f463fef7bf0407497681745494aed96627e6aa733fb862621c530fb07a1bbe83c5bd61c0aea3bdffde8fec1ec5c2dd1875477a40b46359e4c12da8392017faef5f26df8ed2b81707504c4a232b573a17a7304f3391027f697ec6756c9a3dc928251d2dc98ec5130057b113b000a8347173df055db10aba90bc649b12fa87d1e89799f0265b33f1a62fe82d82a0caf2d035a6053db5a641db7dbad8223be8a81cfebdeb6865ce5ac4b358f0b131175a8f27f082880f1cdd6a20aec12ac05dbd864ef3af464cc7893ffd1a893ff95435fec4fb9a3463b1581f98412aad115932862180d07fce170878c5f9d2c29500bc8b843d60acf3f7ec7bd3245d5506eff86d307458b195e2262a0f36908806b4bbb32a3c04d96cc683783b141f7ccf692fc971f579f36427b20725d3924ee650ce78242d2aee61854286c97635eb1a4a6e4b552922aaadfa11eb6a199efa543305c4e9e80600ee5f8b364528650cda03f991834e95042036061751effa6edc4d18fcefc3a89218739f4953493696bf28543db7d59a67b5f2801ca87477ee90363165bd955e7ff831cb6f40764ddd97fa306a8240b7e7fc73a1575d4c82d5373ed2ff581cfb9fb9b340e77207ca6af7570b2f7740d45a24d92090d9e11459380bae0844a9fd29c0d364f0188c9222c9eaf7b172433e91784a508e21769d870e490da0d18514b017fb8e6ad027678185bd24bbec67d3c54fcef0c05da8312f72407453ff8c005d1117137c4159668508f8c0a1f67b2f1ffb4c607c30cfb2e2b510b4698505ea8251d58b0cb1881bda9e5accb')

2048

In [99]:
for i in range(0, 32):
    print('''in_sha256_in_ext_data_'''+str(i)+'''  <= x"00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000";''')

in_sha256_in_ext_data_0  <= x"00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000";
in_sha256_in_ext_data_1  <= x"00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000";
in_sha256_in_ext_data_2  <= x"00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000";
in_sha256_in_ext_data_3  <= x"00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000";
in_sha256_in_ext_data_4  <= x"00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000";
in_sha256_in_ext_data_5  <= x"00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000";
in_sha256_in_ext_data_6  <= x"0000

In [34]:
for i in range(0, 32):
    print('''in_sha256_in_ext_data_'''+str(i) + ''' <= I & x"'''+hex(32 + i)[2:].zfill(8)+ '''" & x"8282" & public_keys('''+str(8191-(256*i))+''' downto '''+str(7936 - (i * 256))+''') & padding;''')

in_sha256_in_ext_data_0 <= I & x"00000020" & x"8282" & public_keys(8191 downto 7936) & padding;
in_sha256_in_ext_data_1 <= I & x"00000021" & x"8282" & public_keys(7935 downto 7680) & padding;
in_sha256_in_ext_data_2 <= I & x"00000022" & x"8282" & public_keys(7679 downto 7424) & padding;
in_sha256_in_ext_data_3 <= I & x"00000023" & x"8282" & public_keys(7423 downto 7168) & padding;
in_sha256_in_ext_data_4 <= I & x"00000024" & x"8282" & public_keys(7167 downto 6912) & padding;
in_sha256_in_ext_data_5 <= I & x"00000025" & x"8282" & public_keys(6911 downto 6656) & padding;
in_sha256_in_ext_data_6 <= I & x"00000026" & x"8282" & public_keys(6655 downto 6400) & padding;
in_sha256_in_ext_data_7 <= I & x"00000027" & x"8282" & public_keys(6399 downto 6144) & padding;
in_sha256_in_ext_data_8 <= I & x"00000028" & x"8282" & public_keys(6143 downto 5888) & padding;
in_sha256_in_ext_data_9 <= I & x"00000029" & x"8282" & public_keys(5887 downto 5632) & padding;
in_sha256_in_ext_data_10 <= I & x"000000

In [49]:
s = "61A5D57D37F5E46BFB7520806B07A1B8000000108383A80DA2194CDF0497BB1AF9F5961D60CAC572B59A9C592ADEFC77C839C5D3D983A69C9D9978AF4E2F075B78AB789364D61B4E94670951054AC6D65A83F2C5A5E48000000000000000000000000000000000000000000000000000000000000000000000000000000002B0"
s1 = s[:len(s)//2]
s2 = s[len(s)//2:]
print(s1)

print()

print(s2)

61A5D57D37F5E46BFB7520806B07A1B8000000108383A80DA2194CDF0497BB1AF9F5961D60CAC572B59A9C592ADEFC77C839C5D3D983A69C9D9978AF4E2F075B

78AB789364D61B4E94670951054AC6D65A83F2C5A5E48000000000000000000000000000000000000000000000000000000000000000000000000000000002B0


In [62]:
for i in range(0, 8):
    print('''in_sha256_in_ext_data_'''+str(i)+''' <= in_sha256_node_'''+str(8+i)+'''(511 downto 0);''')

in_sha256_in_ext_data_0 <= in_sha256_node_8(511 downto 0);
in_sha256_in_ext_data_1 <= in_sha256_node_9(511 downto 0);
in_sha256_in_ext_data_2 <= in_sha256_node_10(511 downto 0);
in_sha256_in_ext_data_3 <= in_sha256_node_11(511 downto 0);
in_sha256_in_ext_data_4 <= in_sha256_node_12(511 downto 0);
in_sha256_in_ext_data_5 <= in_sha256_node_13(511 downto 0);
in_sha256_in_ext_data_6 <= in_sha256_node_14(511 downto 0);
in_sha256_in_ext_data_7 <= in_sha256_node_15(511 downto 0);


In [ ]:
for i in range(0, 32):
    print('''if(q(4 downto 0) = b"'''+bin(i)[2:].zfill(5)+'''") then''')
    if i % 2 == 0:
        print('''\tpath_1 <= out_sha256_out_ext_data_'''+str(i+1)+''';''')
    else :
        print('''\tpath_1 <= out_sha256_out_ext_data_'''+str(i-1)+''';''')
    print("end if;\n")

if(q(4 downto 0) = b"00000") then
	path_1 <= out_sha256_out_ext_data_1;
end if;

if(q(4 downto 0) = b"00001") then
	path_1 <= out_sha256_out_ext_data_0;
end if;

if(q(4 downto 0) = b"00010") then
	path_1 <= out_sha256_out_ext_data_3;
end if;

if(q(4 downto 0) = b"00011") then
	path_1 <= out_sha256_out_ext_data_2;
end if;

if(q(4 downto 0) = b"00100") then
	path_1 <= out_sha256_out_ext_data_5;
end if;

if(q(4 downto 0) = b"00101") then
	path_1 <= out_sha256_out_ext_data_4;
end if;

if(q(4 downto 0) = b"00110") then
	path_1 <= out_sha256_out_ext_data_7;
end if;

if(q(4 downto 0) = b"00111") then
	path_1 <= out_sha256_out_ext_data_6;
end if;

if(q(4 downto 0) = b"01000") then
	path_1 <= out_sha256_out_ext_data_9;
end if;

if(q(4 downto 0) = b"01001") then
	path_1 <= out_sha256_out_ext_data_8;
end if;

if(q(4 downto 0) = b"01010") then
	path_1 <= out_sha256_out_ext_data_11;
end if;

if(q(4 downto 0) = b"01011") then
	path_1 <= out_sha256_out_ext_data_10;
end if;

if(q(4 downto 0) = b"01100

In [7]:
for i in range(0, 34):
    print('''\t\t\tif(unsigned(counter) > 273 and out_get_yy'''+str(i)+'''_busy = '0') then
                y'''+str(i)+'''_done <= '1';
                y'''+str(i)+''' <= out_get_yy'''+str(i)+'''_y;
            else 
                y'''+str(i)+'''_done <= '0';
                y'''+str(i)+''' <= x"0000000000000000000000000000000000000000000000000000000000000000";
\t\t\tend if;\n''')

			if(unsigned(counter) > 273 and out_get_yy0_busy = '0') then
                y0_done <= '1';
                y0 <= out_get_yy0_y;
            else 
                y0_done <= '0';
                y0 <= x"0000000000000000000000000000000000000000000000000000000000000000";
			end if;

			if(unsigned(counter) > 273 and out_get_yy1_busy = '0') then
                y1_done <= '1';
                y1 <= out_get_yy1_y;
            else 
                y1_done <= '0';
                y1 <= x"0000000000000000000000000000000000000000000000000000000000000000";
			end if;

			if(unsigned(counter) > 273 and out_get_yy2_busy = '0') then
                y2_done <= '1';
                y2 <= out_get_yy2_y;
            else 
                y2_done <= '0';
                y2 <= x"0000000000000000000000000000000000000000000000000000000000000000";
			end if;

			if(unsigned(counter) > 273 and out_get_yy3_busy = '0') then
                y3_done <= '1';
                y3 <= out_get_yy3_y;
            e

In [13]:
for i in range(0, 34):
    print('''y'''+str(i)+'''_done <= '1' when unsigned(counter) > 273 and out_get_yy0_busy = '0' else '0';\n''')

y0_done <= '1' when unsigned(counter) > 273 and out_get_yy0_busy = '0' else '0';

y1_done <= '1' when unsigned(counter) > 273 and out_get_yy0_busy = '0' else '0';

y2_done <= '1' when unsigned(counter) > 273 and out_get_yy0_busy = '0' else '0';

y3_done <= '1' when unsigned(counter) > 273 and out_get_yy0_busy = '0' else '0';

y4_done <= '1' when unsigned(counter) > 273 and out_get_yy0_busy = '0' else '0';

y5_done <= '1' when unsigned(counter) > 273 and out_get_yy0_busy = '0' else '0';

y6_done <= '1' when unsigned(counter) > 273 and out_get_yy0_busy = '0' else '0';

y7_done <= '1' when unsigned(counter) > 273 and out_get_yy0_busy = '0' else '0';

y8_done <= '1' when unsigned(counter) > 273 and out_get_yy0_busy = '0' else '0';

y9_done <= '1' when unsigned(counter) > 273 and out_get_yy0_busy = '0' else '0';

y10_done <= '1' when unsigned(counter) > 273 and out_get_yy0_busy = '0' else '0';

y11_done <= '1' when unsigned(counter) > 273 and out_get_yy0_busy = '0' else '0';

y12_done <= '1

In [4]:
for i in range(0, 34):
    print("y"+str(i)+"_done = '1'", end=' and ')

y0_done = '1' and y1_done = '1' and y2_done = '1' and y3_done = '1' and y4_done = '1' and y5_done = '1' and y6_done = '1' and y7_done = '1' and y8_done = '1' and y9_done = '1' and y10_done = '1' and y11_done = '1' and y12_done = '1' and y13_done = '1' and y14_done = '1' and y15_done = '1' and y16_done = '1' and y17_done = '1' and y18_done = '1' and y19_done = '1' and y20_done = '1' and y21_done = '1' and y22_done = '1' and y23_done = '1' and y24_done = '1' and y25_done = '1' and y26_done = '1' and y27_done = '1' and y28_done = '1' and y29_done = '1' and y30_done = '1' and y31_done = '1' and y32_done = '1' and y33_done = '1' and 

In [6]:
for i in range(0, 34):
    print('''
    y'''+str(i)+'''_done <= '1' when unsigned(counter) > 274 and out_get_yy'''+str(i)+'''_busy = '0' else '0';
    y'''+str(i)+''' <= out_get_yy'''+str(i)+'''_y when unsigned(counter) > 274 and out_get_yy'''+str(i)+'''_busy = '0' else x"0000000000000000000000000000000000000000000000000000000000000000";''')


    y0_done <= '1' when unsigned(counter) > 274 and out_get_yy0_busy = '0' else '0';
    y0 <= out_get_yy0_y when unsigned(counter) > 274 and out_get_yy0_busy = '0' else x"0000000000000000000000000000000000000000000000000000000000000000";

    y1_done <= '1' when unsigned(counter) > 274 and out_get_yy1_busy = '0' else '0';
    y1 <= out_get_yy1_y when unsigned(counter) > 274 and out_get_yy1_busy = '0' else x"0000000000000000000000000000000000000000000000000000000000000000";

    y2_done <= '1' when unsigned(counter) > 274 and out_get_yy2_busy = '0' else '0';
    y2 <= out_get_yy2_y when unsigned(counter) > 274 and out_get_yy2_busy = '0' else x"0000000000000000000000000000000000000000000000000000000000000000";

    y3_done <= '1' when unsigned(counter) > 274 and out_get_yy3_busy = '0' else '0';
    y3 <= out_get_yy3_y when unsigned(counter) > 274 and out_get_yy3_busy = '0' else x"0000000000000000000000000000000000000000000000000000000000000000";

    y4_done <= '1' when unsigned(counte

In [5]:
def Cantor_half_reduction(D_1, D_2, f, h):

    a1 = D_1[0]
    a2 = D_2[0]
    
    b1 = D_1[1]
    b2 = D_2[1]
    
    d1, e1, e2 = xgcd(a1, a2)

    d, c1, c2 = xgcd(d1, b1+b2+h)

    s1 = c1*e1
    s2 = c1*e2 
    s3 = c2
    
    a = R((a1*a2) / (d**2))

    b = (R((s1*a1*b2) + (s2*a2*b1) + (s3*(b1*b2 + f))) * d.inverse_mod(a)).mod(a)

    return R(a), R(b)

def Cantor_reduction(D, f, g):
    a = D[0]
    b = D[1]
    

    h = 0
    
    a_prim = R(R(f - (b*h) - (b**2)) / R(a))
#     print(type(a), type(f - (b*h) - (b**2)))
    b_prim = R((-h - b)).mod(a_prim)
#     print("a_prim  of  while = "+str(a_prim))
    deg_a_prim = R(a_prim).degree()
#     print(a_prim, b_prim)
    while deg_a_prim <= g:
#         if deg_a_prim > g:
#             break
        a_prim = R(R(f - (b*h) - (b^2)) / R(a))
        b_prim = R((-h - b)).mod(a_prim)
        
        a = a_prim
        b = b_prim
        print("a_prim = "+str(a_prim))
        deg_a_prim = R(a_prim).degree()
        
#     print("dsdas " + str(a_prim))
    return R(a_prim), R(b_prim)
   
def Montgomery_ladder(D, k, f, g, h):
    R0 = D
    R1 = Cantor_half_reduction(D, D, f, h)
    R1 = Cantor_reduction(R1, f, g)
    print(R0, R1)
    K = bin(k)[3:]
#     try:
    for i in range(0, len(K)):
        print(i)
        if K[i] == '1':
#                 print
            R0 = Cantor_half_reduction(R1, R0, f, h)
            R0 = Cantor_reduction(R0, f, g)
            R1 = Cantor_half_reduction(R1, R1, f, h)
            R1 = Cantor_reduction(R1, f, g)
        else:
            R1 = Cantor_half_reduction(R0, R1, f, h)
            R1 = Cantor_reduction(R1, f, g)
                
            R0 = Cantor_half_reduction(R0, R0, f, h)
            R0 = Cantor_reduction(R0, f, g)
                
#                 R1 = point_add(R0, R1, A, B, q)
#                 R0 = point_double(R0, A, B, q)
    return R0
#     except ZeroDivisionError:
#         return("Punkt w nieskonczonosci")

p = 1048583
F = GF(p)
R.<x> = F[]

X = 1
Y = 0

f = x^5 + 812044*x^4 + 543246*x^3 + 230741*x^2 + 825190*x + 633221
g = 2

C =  HyperellipticCurve(f)                                                 
J = C.jacobian()                                                          
JJ = J(F)
ord = 1099305873793
P = C([X, Y])
D = JJ(P)
h = 0

D_prim = Cantor_half_reduction(D, D, f, h)
D_reducted = Cantor_reduction(D_prim, f, g)

a, b = D_reducted
print(D_prim, type(D_prim[0]), type(D_prim[1]))
print(D_reducted)

Montgomery_ladder(D, 5, f, g, h)

SyntaxError: invalid syntax (<ipython-input-5-c6f20500e7fa>, line 80)